In [1]:

import urllib.request
import json 

categories = {
    "other":"32",
    "bd-illustration":"33",
    "movies":"34",
    "food":"35",
    "geek":"36",
    "video-game":"38",
    "humour":"37",
    "journalism":"39",
    "books":"40",
    "fashion":"41",
    "music":"42",
    "photography":"43",
    "science-technology":"44",
    "performing-arts":"45",
    "sports":"46",
    "vlog":"47",
    "streaming":"52"
}

superheaders = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
               'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
               'Accept-Encoding': 'none',
               'Accept-Language': 'en-US,en;q=0.8',
               'Connection': 'keep-alive'}

def show_categories(categories = categories):
    """
    Show list of admissible values for category 
    """
    for category in categories.keys():
        print(category)
        
# TO REQUEST ONE SINGLE JSON WEB PAGE
def requesting(url, headers = superheaders):
    req = urllib.request.Request(url,headers=headers)
    response= urllib.request.urlopen(req)
    data = response.read()
    encoding = response.info().get_content_charset('utf-8')
    data = json.loads(data.decode(encoding))
    return data

In [2]:
class Comment:
    '''
    When you scrape comments this class help you to print comments and theyr auhor in a pretty format
    '''
    
    def __init__(self,comment):
        self.id = comment['id']
        self.time = comment['created_at']
        self.author = comment['author']['username']
        self.author_id = comment['author']['id']
        def __cleanComment(text):
            import re 
            return re.sub('\n',' ',text)
        self.text = comment['body']
        
    def __repr__(self):
        
        return '\n ' + '\033[1m' + self.author+ '\033[0m' + '\n_____________________________________\n' + self.text + '\n'
   

In [3]:
class Tipper:
    '''
    Subscribed common user of tipeee
    '''
    
    def __init__(self, username):
        self.username = username
        self.tips = list()
    def __repr__(self):
        return self.username
    
    def get_tips(self):
        '''
        See all tips fro this user
        '''
        self.tips = list()
        data = requesting("https://api.tipeee.com/v2.0/users/{}".format(self.username))
        for subscription in data['activeSubscriptions'].keys():
            self.tips.append(data['activeSubscriptions'][subscription]['slug'])
        return self.tips
            
        
        

In [4]:
class Creator:
    
    
    def __init__(self, user):
        
        if type(user) == str: 
            self.username = user
            self.data = requesting("https://api.tipeee.com/v2.0/projects/{}".format(user))
            self.scraped = True
        else: 
            self.data = user
            self.username = user['slug']
            self.scraped = False
            
        self.id = self.data['id']
        self.lang = self.data['lang']
        self.tipperAmount = self.data['parameters']['tipperAmount']
        self.tipperNumber = self.data['parameters']['tipperNumber']
        self.newsNumber =   self.data['parameters']['newsNumber']
        self.subscription =  self.data['parameters']['activateAt']
        self.categories = set(category['slug'] for category in self.data['categories'])
        self.comments = list()
        self.goals = list()
        self.news = list()
        self.tippers = list()
        
        if self.scraped == True:
            self.num_comments = int(self.data['thread']['num_comments'])
            self.num_rewards = len(self.data['rewards'])
            self.num_goals = len(self.data['goals'])
        else :
            self.num_goals = None 
            self.num_rewards = None
            self.num_comments = None
            
    def more_info(self): 
        ''' 
        scrape the creators own web page
        '''
        self = self.__init__(self.username)
        
        
    def visit(self):
        '''
        Open the creator tipeee own web page 
        '''
        import webbrowser
        webbrowser.open_new("https://en.tipeee.com/{}/".format(self.username))
        
    def to_dict(self):
        '''
        Create a dictionary with main information for the creator
        '''
        return {
            "id" : self.id,
            "username" : self.username ,
            "lang" : self.lang,
            #"name" : self.name,
            "tipperAmount": self.tipperAmount,
            "tipperNumber" :self.tipperNumber,
            "newsNumber" : self.newsNumber,
            "subsciption" :   self.subscription,
            "categories" : self.categories,
            "num_comments": self.num_comments,
            "num_goals": self.num_goals,
            "num_rewards": self.num_rewards,
            
        }
    
    
    def get_comments(self):
        ''' 
        Return a list with all comments
        '''
        self.comments = list()
        page = '1'
        while True:
            data = requesting("https://api.tipeee.com/v2.0/threads/project_{}?page={}&perPage=150".format(self.id,page))
            for item in data['items'] : 
                self.comments.append(Comment(item))
            try: page = data['pager']['next']
            except: break
        return self.comments
                
    
    def get_news(self):
        '''
        Return list of all news names
        '''
        self.news = list()
        page = '1'
        while True:
            data = requesting("https://api.tipeee.com/v2.0/projects/{}/news?page={}&perPage=150".format(self.username,page))
            for item in data['items'] : 
                self.news.append(item['name'])
            try: page = data['pager']['next']
            except: break
        return self.news
    
    
    def get_tippers(self):
        self.tippers = list()
        page='1'
        while True:
            data = requesting("https://api.tipeee.com/v2.0/projects/{}/top/tippers?page={}&perPage=150".format(self.username,page))
            for item in data['items'] : 
                try: self.tippers.append(Tipper(item['username_canonical']))
                    except: continue
            try: page = data['pager']['next']
            except: break
        return self.tippers
            
    
    def __repr__(self): return str(self.to_dict())

     
    def __gt__(self, anotherCreator):
        assert(type(anotherCreator) == type(self))
        if self.tipperNumber > anotherCreator.tipperNumber : return True
        else : return False 
        
    def __lt__(self, anotherCreator):
        assert(type(anotherCreator) == type(self))
        if self.tipperNumber < anotherCreator.tipperNumber : return True
        else : return False 
        
    def __eq__(self, anotherAuthor):
        assert(type(anotherCreator) == type(self))
        if self.username == anotherCreator.username : return True
        else : return False 
        

In [5]:

class Creators:
    
    def __init__(self, lang='en'):
        self.lang = lang
        self.scraped =   list()
        self.creators = list()
       
        
    def __iter__(self):
        for elem in self.creators: 
            yield elem['slug']
            
    def __len__(self):
        return len(self.creators)
    
    def __repr__(self):
        return str(self.creators)
        
    def scrape(self, limit=None, category=None, headers=superheaders, lang=None):
        '''
        returns creator in a list 

       Parameters
       ----------
           limit : the number of creators to get, if not specified return all authors.
                Authors are crawled in the order giben by the site 
                N.B. if not specified can take some time 

           category : specifying a category will obtain only authors of that category 
               N.B. run pytipeee.show_categories() to see admissible vlaues for category
       '''    
        
        def __params_setting(category=category, limit=limit, headers = headers, lang=lang):
            if limit != None: assert(type(limit)==int)
            if lang not in ['en','de','fr','es','it']: lang = 'en'
            mode = 'default'
            if category != None:
                if category in categories:
                    category = '&category='+str(categories[category])
                    mode = 'category'
                else:
                    print('wrong value for category: will not be considered')
            else:
                category=''
            return limit, mode, category, headers, lang
        
             # PARAMETHERS SETTING
        limit, mode ,category, headers, lang = __params_setting(category,limit,headers,lang)
        page='1'
        base_url = 'https://api.tipeee.com/v2.0/projects?mode={}&page={}&perPage=150&lang={}{}'
        creators_list = list()
        
            #COLLECTING DATA
        while len(creators_list) < limit:
            data = requesting(base_url.format(mode, page, lang, category), headers) 
            creators_list += data['items']
            try: 
                page = data['pager']['next']
            except: 
                break
        if len(creators_list) >  limit : creators_list = creators_list[:limit]
        
        self.scraped = creators_list
        return creators_list
    
    
    def get_creators(self):
        """
        Transform each scraped eleemtn to an element of the Creator class
        """
        if len(self.scraped) > 0 :
            for item in self.scraped:
                try :
                    self.creators.append(Creator(item))
                except: 
                    print(item)
                    break
                    
   
    
    
    def to_dataframe(self, lang=None):
        """
        return a pandas dataframe 
        
        paramether:
        ____________
            lang: chose the lenguage for categories.
        
        """
            
        if len(self.creators)==0: return 
        import pandas 
        columns = ['id','username','lang','categories','tipperAmount','tipperNumber','newsNumber']
        df = pandas.DataFrame(columns=columns)
        for creator in self.creators:
            df = df.append(creator.to_dict(), ignore_index=True)
        df.set_index('id',inplace=True)   
        
        return df
        
    
   
        

In [6]:
"""======TEST======"""
creators = Creators()
creators_list = creators.scrape(40,'books')
creators.get_creators()
df = creators.to_dataframe()
df.head()

,username,lang,categories,tipperAmount,tipperNumber,newsNumber,num_comments,num_goals,num_rewards,subsciption
id,,,,,,,,,,
235051,la-renaissance-du-plus-puissant-des-dieux-epei...,en,"{art-culture, books}",111,25,305,None,None,None,2020-01-17 05:03:25
237883,panlong,en,"{art-culture, books}",104,22,268,None,None,None,2020-02-24 12:10:27
188944,misty-lament-projec,en,"{bd-illustration, books}",513,17,3,None,None,None,2019-04-16 14:03:33
239615,world-defying-dan-god,en,"{art-culture, books}",22,7,252,None,None,None,2020-03-19 08:04:27
237881,relachez-cette-sorciere,en,"{art-culture, books}",23,5,212,None,None,None,2020-02-24 11:53:30


In [7]:
creators.creators[3].get_comments()

[
  cedric-180
 _____________________________________
 Expansive,
 
  oyatec-suko-tomu
 _____________________________________
 J'adore vraiment son novel.
 Pour vous remercier de le traduire voici un petit tips,
 
  cedric-180
 _____________________________________
 La force absolue]

In [8]:
len(creators.creators[3].get_news())


252

In [9]:
#creators.creators[3].visit()
creators.creators[3].more_info()

In [10]:
creators.creators[3].username

'world-defying-dan-god'

In [11]:
creators.creators[3].get_tippers()

[cedric-180,
 pascalw,
 oyatec-suko-tomu,
 f-nicolas-5d2771,
 jeux-3,
 paul-532,
 cesar-9]

In [62]:
creators.get_creators()
df = creators.to_dataframe()
df.head()

,username,lang,categories,tipperAmount,tipperNumber,newsNumber,num_comments,num_goals,num_rewards,subsciption
id,,,,,,,,,,
235051,la-renaissance-du-plus-puissant-des-dieux-epei...,en,"{art-culture, books}",99,24,304,None,None,None,2020-01-17 05:03:25
237883,panlong,en,"{art-culture, books}",101,21,268,None,None,None,2020-02-24 12:10:27
188944,misty-lament-projec,en,"{books, bd-illustration}",513,17,3,None,None,None,2019-04-16 14:03:33
239615,world-defying-dan-god,en,"{art-culture, books}",22,7,251,None,None,None,2020-03-19 08:04:27
237881,relachez-cette-sorciere,en,"{art-culture, books}",23,5,211,None,None,None,2020-02-24 11:53:30


In [159]:
df = creators.to_dataframe()
df.head()

,username,lang,categories,tipperAmount,tipperNumber,newsNumber,subsciption
id,,,,,,,
235051,la-renaissance-du-plus-puissant-des-dieux-epei...,en,"{books, art-culture}",99,24,304,2020-01-17 05:03:25
237883,panlong,en,"{books, art-culture}",101,21,268,2020-02-24 12:10:27
188944,misty-lament-projec,en,"{books, bd-illustration}",513,17,3,2019-04-16 14:03:33
239615,world-defying-dan-god,en,"{books, art-culture}",22,7,251,2020-03-19 08:04:27
237881,relachez-cette-sorciere,en,"{books, art-culture}",23,5,211,2020-02-24 11:53:30


In [22]:
import re
cleanr = re.compile('<.*?>')
data['longtext'] = re.sub(cleanr, '', data['contentv2']['translations']['en']['html'])

In [23]:
data['longtext']

'Qui je suis et quel type de contenu je produis ?Bienvenue sur mon Tipee et merci de me rejoindre pour cette nouvelle aventure.Depuis pas loin de 15 ans je fais... comment disent les jeunes de nos jours ? Ah oui : de la "création de contenus", voilà. Eh bien pendant plus d\'une décennie j\'ai fait ça sur mon site\xa0ladyteruki.com, et avant ça je le faisais sur les sites d\'autres personnes (feu\xa0SeriesLive.com, par exemple) ou un autre site que j\'avais et que vous avez sûrement oublié,\xa0ladyteruki.free.fr. Et puis maintenant je suis sur Twitch, aussi. Donc voilà maintenant un long, long, looooong moment que je fais des choses gratuitement. J\'écris sur les séries de la planète entière et\xa0l\'histoire de la télévision\xa0(c\'est ce pour quoi vous me connaissez le mieux, probablement)... ou sur d\'autres choses qui me passionnent ; ainsi,\xa0je livestreame aussi des jeux videos.J\'ai toujours évité les revenus publicitaires (il n\'y a aucune pub sur\xa0ladyteruki.com) et les cont

In [64]:
def get_creator(item): 
            return {   
                'id' :           item['id'],
                'username' :     item['slug'],
                'lang' :         item['lang'],
                'photo_link' :   item['avatar']['filename'] ,
                'tipperAmount' : item['parameters']['tipperAmount'],
                'tipperNumber' : item['parameters']['tipperNumber'],
                'newsNumber' :   item['parameters']['newsNumber'],
                'subsciption' :  item['parameters']['activateAt'],
                'categories' :   [category['slug'] for category in item['categories']]
            }

In [65]:
print(get_creator(data))

{'id': 177029, 'username': 'ladyteruki', 'lang': 'en', 'photo_link': '202101015fee7bd73be18.png', 'tipperAmount': '365', 'tipperNumber': '26', 'newsNumber': '220', 'subsciption': '2018-12-26 22:34:00', 'categories': ['art-culture', 'movies']}


In [3]:
url ="https://twitter.com/i/api/2/notifications/all.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&send_error_codes=true&simple_quoted_tweet=true&count=40&cursor=DAABDAABCgABAAAAAF7zDDQIAAIAAAABCAADSQ3bEQgABHQ6AFwACwACAAAAC0FYaHZFTUlURG1VAAA&ext=mediaStats%2ChighlightedLabel"

In [4]:
data = requesting(url)

HTTPError: HTTP Error 400: Bad Request